In [1]:
import pygame
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

#USER INPUT
Dim = 4
PlayerOneisHuman = False 
PlayerTwoisHuman = False
NumberOfGames = 4

#global stuff
Wid = 1024
Hig = Wid*0.5
red = (255,0,0)
blue = (0,0,255)
empty = (85,85,85)
counted = (170,170,170)
redrim = (125,0,0)
bluerim = (0,0,125)


FULLGAME = []
FULLZUGS = []
#moveCounter = 0

X=[]
y=[]

#safety = 0
DEBUG = False
COUNT = True

pygame 2.5.1 (SDL 2.28.2, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\Office\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def debug(stuff):
    if DEBUG:
        print(stuff)
def C(c):
    if COUNT:
        print(c)

In [3]:
class Peghole:
    def __init__(self,idex,jdex, colour, window):
        self.i = idex
        self.j = jdex
        self.color = colour
        self.canvas = window
        self.scanned = False
    def show(self, xpos, ypos, size):
        #if self.scanned == False : 
            pygame.draw.circle(self.canvas, self.color , ( xpos,ypos ) , size)

In [4]:
class agent:
    def __init__(self, dimension, width, height , window, name1, name2, cyborg1 , cyborg2):
        self.Dim = dimension
        self.screenwidth = width
        self.screenheight = height
        self.canvas = window
        self.player = [name1,name2]
        self.turn = False
        self.playerColors = [red, blue]
        self.BGCOL = [redrim, bluerim]
        self.Pegfield = np.empty(self.Dim*self.Dim, dtype=object)
        self.KeyInput = ""
        self.Pegroad = []
        self.Neighbours = []
        self.win = False
        self.isHuman = [cyborg1 , cyborg2] # if cyborg - player is Human! - if False computer
        self.PegInit()
    def PegInit(self):
        self.Pegfield = [[Peghole(i,j,empty,self.canvas) for i in range(Dim)] for j in range(Dim)]
    def show(self):
        Rx = self.screenwidth/(2*(self.Dim))
        Ry = Rx*0.5
        for i in range(self.Dim):
            for j in range(self.Dim):
                x = Rx*(1+i+j)
                y = self.screenheight*0.5+Ry*(i-j)
                self.Pegfield[i][j].show(x,y,Ry*0.5)        
    def switchPlayerColors(self):
        self.player = self.player[::-1]
    def nextTurn(self):
        self.turn = not self.turn
    def valid(self, ex, why):
        if ex >= 0 and ex < Dim and why >= 0 and why < Dim and self.Pegfield[ex][why].color == self.playerColors[self.turn] and self.Pegfield[ex][why].scanned == False:
            self.Pegroad.append((ex,why))
            self.Neighbours.append((ex,why))
            self.Pegroad = list(dict.fromkeys(self.Pegroad))
            self.Neighbours = list(dict.fromkeys(self.Neighbours))
            self.Pegfield[ex][why].scanned == True
            print(self.Pegroad)
    def checkForWin(self) :
        for i in range(Dim):
            for j in range(Dim):
                self.Pegfield[i][j].scanned = False
        self.Neighbours = []
        self.Pegroad = []
        if self.turn:
            for j in range(Dim) :
                if self.Pegfield[0][j].color == self.playerColors[self.turn]:
                    self.Neighbours.append((0,j))
                    self.Pegroad.append((0,j))
                self.Pegfield[0][j].scanned = True   
        else : 
            for i in range(Dim) : 
                if self.Pegfield[i][0].color == self.playerColors[self.turn]:
                    self.Neighbours.append((i,0))
                    self.Pegroad.append((i,0))
                self.Pegfield[i][0].scanned = True
        #while self.Neighbours:
        self.scanNeighbours()
        self.Perculation()
    def scanNeighbours(self) :
        while self.Neighbours:
            self.valid(self.Neighbours[0][0]+1,self.Neighbours[0][1]  )
            self.valid(self.Neighbours[0][0]-1,self.Neighbours[0][1]  )
            self.valid(self.Neighbours[0][0]  ,self.Neighbours[0][1]+1)
            self.valid(self.Neighbours[0][0]  ,self.Neighbours[0][1]-1)
            self.valid(self.Neighbours[0][0]+1,self.Neighbours[0][1]-1)
            self.valid(self.Neighbours[0][0]-1,self.Neighbours[0][1]+1)
            self.Pegfield[self.Neighbours[0][0]][self.Neighbours[0][1]].scanned = True
            self.Neighbours.pop(0)
    def Perculation(self) :
        debug("Perculation()")
        for P in self.Pegroad:
            debug(P)
        for PR in self.Pegroad:
            if self.turn:
                if PR[0] == Dim-1:
                    self.win = True
                    print("player " , self.playerColors[self.turn] , " won")
            else :
                if PR[1] == Dim-1:
                    self.win = True
                    print("player " , self.playerColors[self.turn] , " won")

In [5]:
#AI No0:
class puter:
    def __init__(self, Spielfeld):
        self.Brett = Spielfeld
    def move(self,myTurn):
        if myTurn:
            coordinate = np.random.randint(Dim*Dim-1)
            while self.Brett[coordinate].color == red or self.Brett[coordinate].color == blue :
                coordinate = np.random.randint(Dim*Dim-1)
            return coordinate
#    def randMove(self,myTurn):
#        spit = np.random.randint(Dim*Dim-1)
#        while Field.Pegstore[spit].color is not empty :
#            spit = np.random.randint(Dim*Dim-1)
#        return spit

In [6]:
def AIField():
    myField = []
    entry = 0
    for P in range(Dim*Dim):
        if judge.Pegfield[spoti][spotj].color == judge.playerColors[judge.turn]:
            entry = 1
        elif judge.Pegfield[spoti][spotj].color == judge.playerColors[not judge.turn]:
            entry = -1
        else :
            entry = 0
        myField.append(entry)
    FULLGAME.append(myField)
#    moveCounter += 1
#    print(moveCounter)
#print(myfield)
   # return myField

In [8]:
AInotShown = True
pygame.init()
background_colour = (100,200,200)
(width, height) = (Wid, Hig)
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Tutorial 1')
screen.fill(background_colour)
pygame.draw.polygon(screen, (bluerim), ((0,Hig*0.5),(Wid*0.5,0),(0,0)))
pygame.draw.polygon(screen, (bluerim), ((Wid,Hig),(Wid,Hig*0.5),(Wid*0.5,Hig)))
pygame.draw.polygon(screen, (redrim), ((0,Hig*0.5),(0,Hig),(Wid*0.5,Hig)))
pygame.draw.polygon(screen, (redrim), ((Wid,0),(Wid,Hig*0.5),(Wid*0.5,0)))
#pygame.display.flip()
running = True

#Field = Board(Dim, Wid, Hig, screen)
judge = agent(Dim, Wid, Hig, screen, "A", "B", PlayerOneisHuman,PlayerTwoisHuman)
#Peter = puter(judge)

#For visualizing where the arrow keys have gotten you
#spot = 0
spoti = 0
spotj = 0
colStore = empty
#
FULLGAME = []
AIField()
while running:
    for event in pygame.event.get():
        C(11)
        if event.type == pygame.QUIT:
            running = False
#navigating with arrow Keys - each arrow key handles a lot of the logic, to not interfere with other keys
        if judge.isHuman[judge.turn] : 
            if event.type == pygame.KEYDOWN and judge.win == False:
                if event.key == pygame.K_UP:
                    judge.Pegfield[spoti][spotj].color = colStore     # recolour the previously highlighted peg correctly
                    spoti -= 1                                       # move cursor
                    if spoti < 0:
                        spoti += 1
                    colStore = judge.Pegfield[spoti][spotj].color     # store actual colour of selected peg
                    judge.Pegfield[spoti][spotj].color = counted      # highlight selected beg
                elif event.key == pygame.K_DOWN:
                    judge.Pegfield[spoti][spotj].color = colStore     # recolour the previously highlighted peg correctly
                    spoti += 1                                       # move cursor
                    if spoti == Dim:
                        spoti -= 1
                    colStore = judge.Pegfield[spoti][spotj].color     # store actual colour of selected peg
                    judge.Pegfield[spoti][spotj].color = counted      # highlight selected beg
                elif event.key == pygame.K_RIGHT:
                    judge.Pegfield[spoti][spotj].color = colStore     # recolour the previously highlighted peg correctly
                    spotj += 1                                       # move cursor
                    if spotj == Dim:
                        spotj -= 1
                    colStore = judge.Pegfield[spoti][spotj].color     # store actual colour of selected peg
                    judge.Pegfield[spoti][spotj].color = counted      # highlight selected beg
                elif event.key == pygame.K_LEFT:
                    judge.Pegfield[spoti][spotj].color = colStore     # recolour the previously highlighted peg correctly
                    spotj -= 1                                       # move cursor
                    if spotj < 0:
                        spotj += 1
                    colStore = judge.Pegfield[spoti][spotj].color     # store actual colour of selected peg
                    judge.Pegfield[spoti][spotj].color = counted      # highlight selected beg
                if event.key == pygame.K_RETURN:
                    if colStore == empty:
                        #AIField()
                        FULLZUGS.append((spoti,spotj))
                        colStore = judge.playerColors[judge.turn]
                        judge.Pegfield[spoti][spotj].color = colStore
                        #AIField()
                        judge.checkForWin()
                        if not judge.win :
                            judge.nextTurn()
        elif judge.win == False:
#            print("it moves")
            C(0)
            judge.show()
            pygame.display.update()
#            print("going to sleep")
            C(1)
            time.sleep(1)
            pygame.event.post(pygame.event.Event(pygame.USEREVENT, {}))
            C(2)
            #FULLZUGS.append((spoti,spotj))
            spoti = np.random.randint(Dim-1)
            spotj = np.random.randint(Dim-1)
            
#            print("before while")
            while judge.Pegfield[spoti][spotj].color is not empty :
#                print("try : (" , spoti , "," , spotj , ")" )
                spoti = np.random.randint(Dim)
                spotj = np.random.randint(Dim) 
            C(3)
            judge.Pegfield[spoti][spotj].color = judge.playerColors[judge.turn]
            C(4)
            colStore = judge.Pegfield[spoti][spotj].color
            C(5)#AIField()
            judge.checkForWin()
            C(6)
            if not judge.win :
                judge.nextTurn()
            C(7)
            judge.show()
            pygame.display.update()
            C(8)
    judge.show()
    pygame.draw.circle(screen, judge.playerColors[judge.turn] , (10,10) , 10)
    C(9)
    if judge.win :
        screen.fill(judge.BGCOL[judge.turn] )        
        pygame.draw.polygon(screen, (bluerim), ((0,Hig*0.5),(Wid*0.5,0),(0,0)))
        pygame.draw.polygon(screen, (bluerim), ((Wid,Hig),(Wid,Hig*0.5),(Wid*0.5,Hig)))
        pygame.draw.polygon(screen, (redrim), ((0,Hig*0.5),(0,Hig),(Wid*0.5,Hig)))
        pygame.draw.polygon(screen, (redrim), ((Wid,0),(Wid,Hig*0.5),(Wid*0.5,0)))
        judge.show()    
        pygame.draw.circle(screen, judge.playerColors[judge.turn] , (10,10) , 10)
        pygame.display.update()
        if AInotShown : 
#            print(FULLGAME)
#            print(" / ")
#            print(FULLZUGS)
            #X.append(FULLGAME)
            #y.append(FULLZUGS)
            X = FULLGAME
            y = FULLZUGS
#            print("X")
#            print(X)
#            print("y")
#            print(y)
            AInotShown = False
        print("RUNNOT")
        #running = False
    #judge.show()
    pygame.display.update()
    C(10)
pygame.quit()


9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10
9
10


KeyboardInterrupt: 

In [ ]:
##ABOVE END OF ENGINE - BELOW START PROGRAMMING

In [ ]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [              
        ### START CODE HERE ###
        tf.keras.Input(shape=(Dim*Dim,)),
        Dense(25, activation = 'relu', name = "L1"),
        Dense(15, activation = 'relu', name = "L2"),
        Dense(Dim*Dim, activation = 'linear', name = "L3"),
    
    ], name = "my_model"
)


model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)


history = model.fit(
    X,y,
    epochs=40
)

In [ ]:
prediction = model.predict([[0, 0, 0, 0, 1, 0, 0, -1, 0]])
print(prediction[0])
np.argmax(prediction)
#max = 0
#stelle = 
#for m in range(prediction[0]) :
#    if predm > max :
#        max = m


In [ ]:
model.summary()

In [ ]:
L = [1,2,3]
LL = []
LL.append(L)
print(LL)

In [ ]:
print(np.random.randint(5))

In [ ]:
matrix = [[]]
print(matrix)
I = 1
J = I
for i in range(I):
    for j in range(J):
        matrix[i][j].append(i+I*j)
#print(matrix)

In [ ]:
D = 5
#Field.
Mat = [[i+D*j for i in range(D)] for j in range(D)]
print(Mat)
print(Mat[1][2])

In [ ]:
List = []
Lost = []
Last = [[0]*4]*4
print(Last)
for z in range(8):
    List.append((np.random.randint(4),np.random.randint(4)))
print(List)
while List:
    print("TUPEL = (",List[0][0]+1,",",List[0][1],")")
    Lost.append(List.pop(0))
print(Lost)


In [ ]:
abort = True
abort = abort*True*True*False
print(abort)

In [ ]:
##OLD VERSION
def checkForWin(self) :
        print("check for " , self.playerColors[self.turn])
        Neighbours = []
        self.Pegroad = []
        if self.turn:
            for j in range(Dim) :
                if self.Pegfield[0][j].color == self.playerColors[self.turn]:
                    Neighbours.append((0,j))
                #self.Pegfield[0][j].scanned = True   
        else : 
            for i in range(Dim) : 
                if self.Pegfield[i][0].color == self.playerColors[self.turn]:
                    self.Pegroad.append((i,0))
                #self.Pegfield[i][0].scanned = True
        print("initial Pegroad is " , self.Pegroad)
        while self.Pegroad and self.win == False:
            self.Perculation()
            self.scanNeighbours()
            self.Perculation()
        for i in range(Dim):
            for j in range(Dim):
                self.Pegfield[i][j].scanned = False
    def scanNeighbours(self) :
        print("start Neighbours")
        print("     Pegroad is: " , self.Pegroad) 
        Neighbours = []
        while self.Pegroad and self.win == False:
            print("RoadPeg for Neighbours is scanned = " , self.Pegfield[self.Pegroad[0][0]][self.Pegroad[0][1]].scanned)  
            if self.Pegfield[self.Pegroad[0][0]][self.Pegroad[0][1]].scanned == False:
                print("   append Neighbours")
                Neighbours.append((self.Pegroad[0][0]+1,self.Pegroad[0][1]  ))
                Neighbours.append((self.Pegroad[0][0]-1,self.Pegroad[0][1]  ))
                Neighbours.append((self.Pegroad[0][0]  ,self.Pegroad[0][1]+1))
                Neighbours.append((self.Pegroad[0][0]  ,self.Pegroad[0][1]-1))
                Neighbours.append((self.Pegroad[0][0]+1,self.Pegroad[0][1]-1))
                Neighbours.append((self.Pegroad[0][0]-1,self.Pegroad[0][1]+1))
                self.Pegfield[self.Pegroad[0][0]][self.Pegroad[0][1]].scanned = True
            print("self.Pegroad[0] = ", self.Pegroad[0])
            self.Pegroad.pop(0) #print("popRoad to empty: " , self.Pegroad.pop(0))
        print("     Neighbours is: " , Neighbours)     
        while Neighbours and self.win == False:
            print("entering for-loop:")
            print("N for if1 = (" , Neighbours[0][0] , "," , Neighbours[0][1])
            if Neighbours[0][0] >= 0 and Neighbours[0][0] <= Dim-1 and Neighbours[0][1] >= 0 and Neighbours[0][1] <= Dim-1:
                print("   -if: N is in range yes")
                if self.Pegfield[Neighbours[0][0]][Neighbours[0][1]].scanned == False and self.Pegfield[Neighbours[0][0]][Neighbours[0][1]].color == self.playerColors[self.turn]:
                    self.Pegroad.append(Neighbours[0])
                    print("   -if: Peg not scanned and right color")
                    self.Perculation()
                    self.Pegroad.pop(0)
                else:
                    print("   -else Peg already scanned or wrong color")
                    Neighbours.pop(0) #print("pop to void: ", Neighbours.pop(0))
            else:         
                print("   -else1: N is in range no")
                Neighbours.pop(0)#print("pop to void: ",Neighbours.pop(0))
        print("after for-loop")
        print("last neighbours are: " , Neighbours)
        #self.Pegroad = []
    def Perculation(self) :
        print("perculation")
        proceed = True
        print("Pegroad is: ",self.Pegroad)
        for s in self.Pegroad:
            abort = self.Pegfield[s[0]][s[1]].scanned*proceed
            print("proceed = " , self.Pegfield[s[0]][s[1]].scanned*proceed )
            print("(",[s[0]],",",[s[1]],")")
        while self.Pegroad and self.win == False and not proceed:
            print("perculations Road is " , self.Pegroad)
            if self.turn:
#                print("Turn = " , self.playerColors[self.turn] , " and, s[0] = " , s[0] , "while Dim-1 = " , Dim-1)
                if self.Pegroad[0][0] == Dim-1:
                    self.win = True
                    print("player " , self.playerColors[self.turn] , " won")
            else :
#                print("Turn = " , self.playerColors[self.turn] , " and, s[1] = " , s[1] , "while Dim-1 = " , Dim-1)
                if self.Pegroad[0][1] == Dim-1:
                    self.win = True
                    print("player " , self.playerColors[self.turn] , " won")
            self.Pegfield[self.Pegroad[0][0]][self.Pegroad[0][1]].scanned = True